In [89]:
import os
import math
import warnings
import logging
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
import numpy as np
import pandas as pd
from scipy import stats
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
import lightgbm as lgb


warnings.filterwarnings("ignore")

In [ ]:
# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

@dataclass
class Config:
    data_root: str = ""
    out_root: str = "features_out_data_feature_selection"
    splits: Tuple[int, int] = (1, 20)
    valid_filters: Tuple[str, ...] = ("u", "g", "r", "i", "z", "y")

    # Cleaning
    dropna_cols: Tuple[str, ...] = ("object_id", "Time (MJD)", "Flux", "Flux_err", "Filter")
    min_fluxerr: float = 0.0
    merge_duplicates: bool = True

    # Time features
    snr_det_threshold: float = 3.0
    snr_strong_threshold: float = 5.0
    
    # Color features
    color_max_dt_days: float = 1.0

    # Shape fitting (NOW ENABLED!)
    do_shape_fitting: bool = True  
    shape_fit_min_points: int = 6  
    shape_fit_min_peak_snr: float = 5.0  
    shape_fit_per_filter: bool = True  


    # Missing handling
    do_imputation: bool = True

    # Feature selection
    do_feature_selection: bool = True  
    corr_threshold: float = 0.95
    rf_top_k: int = 150
    random_state: int = 42
    
    # Performance
    use_multiprocessing: bool = False 
    n_jobs: int = 4

In [ ]:
# =========================
# COSMOLOGY HELPERS - OPTIMIZED
# =========================

C_KM_S = 299792.458

_DL_CACHE = {}

def E_z(z: np.ndarray, Om0: float = 0.3) -> np.ndarray:
    """E(z)=H(z)/H0 for flat LCDM."""
    Ol0 = 1.0 - Om0
    return np.sqrt(Om0 * (1 + z)**3 + Ol0)

def luminosity_distance_Mpc_fast(z: np.ndarray, H0: float = 70.0, Om0: float = 0.3) -> np.ndarray:
    z = np.asarray(z, dtype=float)
    out = np.full_like(z, np.nan, dtype=float)
    
    mask = np.isfinite(z) & (z >= 0)
    if not mask.any():
        return out
    
    z_valid = z[mask]
    z_unique = np.unique(z_valid)
    
    # Compute for unique z values only
    for zz in z_unique:
        cache_key = f"{zz:.6f}_{H0}_{Om0}"
        
        if cache_key not in _DL_CACHE:
            if zz == 0:
                _DL_CACHE[cache_key] = 0.0
            else:
                grid = np.linspace(0.0, zz, 256)  # Reduced from 512
                integral = np.trapz(1.0 / E_z(grid, Om0=Om0), grid)
                d_c = (C_KM_S / H0) * integral
                d_l = (1.0 + zz) * d_c
                _DL_CACHE[cache_key] = d_l
        
        # Fill all matching z values
        z_mask = (z_valid == zz)
        out[np.where(mask)[0][z_mask]] = _DL_CACHE[cache_key]
    
    return out



In [92]:
def absolute_mag_from_flux_uJy(flux_uJy: np.ndarray, d_L_Mpc: np.ndarray) -> np.ndarray:
    """Convert flux to absolute magnitude."""
    flux = np.asarray(flux_uJy, dtype=float)
    dL = np.asarray(d_L_Mpc, dtype=float)
    out = np.full_like(flux, np.nan, dtype=float)

    mask = np.isfinite(flux) & (flux > 0) & np.isfinite(dL) & (dL > 0)
    if not mask.any():
        return out

    m_ab = 23.9 - 2.5 * np.log10(flux[mask])
    DM = 5.0 * np.log10(dL[mask]) + 25.0
    out[mask] = m_ab - DM
    return out

In [ ]:
# =========================
# STATISTICAL FEATURES 
# =========================

def stats_features_per_filter_enhanced(df: pd.DataFrame, value_col: str, cfg: Config) -> pd.DataFrame:

    g = df.groupby(["object_id", "Filter"])[value_col]

    def mad(x):
        """Median Absolute Deviation"""
        return np.median(np.abs(x - np.median(x)))
    
    def beyond_1std(x):
        """Fraction of points beyond 1 sigma"""
        if len(x) < 3:
            return 0.0
        m = np.mean(x)
        s = np.std(x)
        return float(np.mean(np.abs(x - m) > s))

    feats = g.agg(
        mean="mean",
        std="std",
        median="median",
        min="min",
        max="max",
        q10=lambda x: np.nanpercentile(x, 10),
        q90=lambda x: np.nanpercentile(x, 90),
        skew=lambda x: stats.skew(x, nan_policy='omit'),
        kurt=lambda x: stats.kurtosis(x, nan_policy='omit'),
        mad=mad,
        amplitude=lambda x: np.nanmax(x) - np.nanmin(x),
        beyond_1std=beyond_1std
    ).reset_index()

    # Pivot to wide
    wide = feats.pivot(index="object_id", columns="Filter")
    wide.columns = [f"{f}_{stat}_{value_col}" for stat, f in wide.columns]
    wide = wide.reset_index()
    
    return wide

In [94]:
def count_features_per_filter(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    """Count observations per filter."""
    g = df.groupby(["object_id", "Filter"]).size().reset_index(name="n_obs")
    wide = g.pivot(index="object_id", columns="Filter", values="n_obs")
    wide.columns = [f"n_obs_{f}" for f in wide.columns]
    wide = wide.fillna(0).reset_index()
    return wide

In [ ]:
def snr_features(df: pd.DataFrame, flux_col: str, cfg: Config) -> pd.DataFrame:
    """SNR-based features."""
    out = df.copy()
    err_col = "Flux_err"
    out["SNR"] = out[flux_col].values / out[err_col].values

    grp = out.groupby("object_id")
    feats = pd.DataFrame({
        "snr_max": grp["SNR"].max(),
        "snr_mean": grp["SNR"].mean(),
        "snr_median": grp["SNR"].median(),  
        "snr_std": grp["SNR"].std(),
        "n_snr_gt3": grp.apply(lambda x: int(np.sum(x["SNR"].values > cfg.snr_det_threshold))),
        "n_snr_gt5": grp.apply(lambda x: int(np.sum(x["SNR"].values > cfg.snr_strong_threshold))),
        "frac_snr_gt3": grp.apply(lambda x: float(np.mean(x["SNR"].values > cfg.snr_det_threshold))),
        "frac_snr_gt5": grp.apply(lambda x: float(np.mean(x["SNR"].values > cfg.snr_strong_threshold))),
    }).reset_index()
    
    return feats

In [ ]:
def overall_time_features_enhanced(df: pd.DataFrame, flux_col: str, cfg: Config) -> pd.DataFrame:
    # 1. Chuẩn bị dữ liệu
    err_col = "Flux_err"
    tmp = df.copy()
    tmp["SNR"] = tmp[flux_col] / (tmp[err_col] + 1e-6)
    
    # Sort trước để tính toán chính xác
    tmp = tmp.sort_values(["object_id", "Time (MJD)"])
    grp = tmp.groupby("object_id")

    # =========================================================
    # PHẦN 1: VECTORIZED AGGREGATIONS
    # =========================================================
    
    aggs = grp["Time (MJD)"].agg(['min', 'max'])
    aggs.columns = ['t_min', 't_max']
    
    # Tính duration thủ công (An toàn hơn dùng ptp)
    aggs['time_span_days'] = aggs['t_max'] - aggs['t_min']
    
    # 1.2 Peak Time & Peak Flux
    idx_max = grp[flux_col].idxmax()
    peak_info = tmp.loc[idx_max, ["object_id", "Time (MJD)", flux_col, "SNR"]].set_index("object_id")
    peak_info.columns = ["t_peak_mjd", "peak_flux", "peak_snr"]
    
    # Merge lại
    feats = aggs.join(peak_info)
    
    # 1.3 Time Window Features
    feats["pre_peak_duration"] = feats["t_peak_mjd"] - feats["t_min"]
    feats["post_peak_duration"] = feats["t_max"] - feats["t_peak_mjd"]
    feats["rise_fall_ratio"] = feats["pre_peak_duration"] / (feats["post_peak_duration"] + 1e-5)

    # =========================================================
    # PHẦN 2: COMPLEX FEATURES 
    # =========================================================

    def calc_complex_features(g):
        t = g["Time (MJD)"].values
        f = g[flux_col].values
        snr = g["SNR"].values
        
        res = {}
        
        # --- A. Cadence ---
        if len(t) >= 2:
            dt = np.diff(t)
            res['cadence_median'] = np.median(dt)
            res['cadence_max'] = np.max(dt)
        else:
            res['cadence_median'] = np.nan
            res['cadence_max'] = np.nan
            
        # --- B. Peak Counting ---
        n_peaks = 0
        if len(t) >= 5:
            try:
                h = f.mean() + f.std()
                p = f.std() * 0.5
                peaks, _ = find_peaks(f, height=h, prominence=p)
                n_peaks = len(peaks)
            except:
                n_peaks = 0
        res['n_peaks'] = n_peaks

        # --- C. Robust Rise/Decay Rate ---
        det_mask = snr > 3.0 
        
        if det_mask.sum() >= 4:
            t_det = t[det_mask]
            f_det = f[det_mask]
            # Lấy index đỉnh tương đối trong tập detection
            idx_peak_det = np.argmax(f_det)
            t_peak = t_det[idx_peak_det]
            
            mask_rise = t_det <= t_peak
            mask_decay = t_det >= t_peak
            
            def robust_slope(tx, fx):
                if len(tx) < 2: return np.nan
                return (fx[-1] - fx[0]) / (tx[-1] - tx[0] + 1e-5)

            res['rise_rate'] = robust_slope(t_det[mask_rise], f_det[mask_rise])
            res['decay_rate'] = robust_slope(t_det[mask_decay], f_det[mask_decay])
        else:
            res['rise_rate'] = np.nan
            res['decay_rate'] = np.nan

        return pd.Series(res)

    complex_feats = grp.apply(calc_complex_features)
    
    # =========================================================
    # PHẦN 3: KẾT HỢP
    # =========================================================
    final_feats = feats.join(complex_feats)
    
    return final_feats.reset_index()

In [ ]:
def rise_decay_features_powerlaw(df: pd.DataFrame, flux_col: str, cfg: Config) -> pd.DataFrame:
    err_col = "Flux_err"
    tmp = df.copy()
    
    # 1. Pre-calculate SNR & Peak info (Vectorized - Fast)
    tmp["SNR"] = tmp[flux_col] / (tmp[err_col] + 1e-6)

    # Tìm index của đỉnh sáng nhất cho mỗi object
    peak_idx = tmp.groupby("object_id")[flux_col].idxmax()
    
    # Tạo bảng lookup đỉnh
    peak_info = tmp.loc[peak_idx, ["object_id", "Time (MJD)", "SNR"]].set_index("object_id")
    peak_info.columns = ["t_peak", "peak_snr"] # Đổi tên cho gọn

    # Merge đỉnh vào bảng chính
    tmp = tmp.merge(peak_info, on="object_id", how="left")
    
    # Tính thời gian tương đối
    tmp["t_rel"] = tmp["Time (MJD)"] - tmp["t_peak"]

    # Hàm tính R2 thủ công (an toàn với NaN)
    def compute_r2(y, yhat):
        ss_res = np.sum((y - yhat) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        if ss_tot < 1e-9: return 0.0 # Tránh chia 0 nếu đường thẳng nằm ngang
        return 1.0 - ss_res / ss_tot

    def per_obj(g: pd.DataFrame) -> pd.Series:
        # Lấy thông tin đỉnh từ dòng đầu tiên 
        f_peak = g[flux_col].max()
        peak_snr = g["peak_snr"].iloc[0]

        # --- 1. RISE PHASE ---
        rise_time = np.nan
        # Lấy dữ liệu trước đỉnh
        before = g[(g["t_rel"] < 0) & (g[flux_col] > 0)].sort_values("t_rel")

        if len(before) >= 7 and f_peak > 0: # MIN_RISE_POINTS = 7
            f = before[flux_col].values
            t = before["t_rel"].values

            # Tìm mốc 10% và 90% độ sáng đỉnh
            f10, f90 = 0.1 * f_peak, 0.9 * f_peak
            
            # Tìm index gần nhất
            i10 = np.argmin(np.abs(f - f10))
            i90 = np.argmin(np.abs(f - f90))

            if t[i10] < t[i90]:
                rise_time = abs(t[i90] - t[i10])

        # --- 2. DECAY PHASE ---
        pl_slope = pl_r2 = exp_r2 = decay_discrimination = np.nan
        decay_reliable = 0
        
        # Lấy dữ liệu sau đỉnh (Decay)
        # Cửa sổ 400 ngày là hợp lý cho TDE
        after = g[
            (g["t_rel"] > 5) &           # Bỏ qua 5 ngày đầu quanh đỉnh (thường nhiễu)
            (g["t_rel"] < 400) & 
            (g[flux_col] > 0) & 
            (g["SNR"] > 3)               # Chỉ fit điểm tin cậy
        ].sort_values("t_rel")

        if len(after) >= 5: # MIN_DECAY_POINTS = 5
            t_fit = after["t_rel"].values
            f_fit = after[flux_col].values
            e_fit = after[err_col].values

            # Trọng số cho fit (Inverse Variance)
            weights = (f_fit / (e_fit + 1e-6)) ** 2
            log_f = np.log(f_fit)

            try:
                # A. Power-law Fit (Linear on log-log)
                # log(F) = a + b * log(t)
                log_t = np.log(t_fit)
                
                # Polyfit bậc 1
                b_pl, a_pl = np.polyfit(log_t, log_f, 1, w=np.sqrt(weights))
                
                # Tính R2
                log_f_hat_pl = a_pl + b_pl * log_t
                pl_r2 = compute_r2(log_f, log_f_hat_pl)
                pl_slope = b_pl

                # B. Exponential Fit (Linear on semi-log)
                # log(F) = a + b * t
                b_exp, a_exp = np.polyfit(t_fit, log_f, 1, w=np.sqrt(weights))
                
                # Tính R2
                log_f_hat_exp = a_exp + b_exp * t_fit
                exp_r2 = compute_r2(log_f, log_f_hat_exp)

                # C. Discrimination Ratio
                if exp_r2 > 0.01: # Tránh chia số quá nhỏ
                    decay_discrimination = pl_r2 / exp_r2
                else:
                    decay_discrimination = 999.0 # Power law thắng tuyệt đối

                # D. Reliability Check
                # Đáng tin nếu Peak SNR cao VÀ fit tốt
                decay_reliable = int(
                    peak_snr > 10 and
                    (pl_r2 > 0.6 or exp_r2 > 0.6)
                )

            except (np.linalg.LinAlgError, ValueError, TypeError):
                pass
            except Exception:
                pass

        return pd.Series({
            "rise_time_10_90": rise_time,
            "pl_slope": pl_slope,
            "pl_r2": pl_r2,
            "exp_r2": exp_r2,
            "decay_discrimination": decay_discrimination,
            "decay_reliable": decay_reliable,
            # TDE chuẩn lý thuyết: slope ~ -5/3 (-1.67)
            "pl_slope_is_tde": int(-2.2 < pl_slope < -1.2) if np.isfinite(pl_slope) else 0
        })

    feats = tmp.groupby("object_id").apply(per_obj).reset_index()
    return feats

In [ ]:
def color_features_unified(df: pd.DataFrame, flux_col: str, cfg: Config) -> pd.DataFrame:
    tmp = df.copy()
    
    err_col = "Flux_err"
    if "SNR" not in tmp.columns:
        tmp["SNR"] = tmp[flux_col] / (tmp[err_col] + 1e-6)
    
    # Lọc dữ liệu: SNR > 3 và Flux > 0
    valid_mask = (tmp["SNR"] > 3) & (tmp[flux_col] > 0)
    tmp = tmp[valid_mask].copy()

    # Convert Flux -> AB Magnitude
    tmp["mag"] = -2.5 * np.log10(tmp[flux_col]) + 23.9
    
    # 2. Định nghĩa các cặp màu
    # Kết hợp cả cặp liền kề (cho SN) và cặp xa (u-z cho TDE)
    color_pairs = [
        ('u', 'g'), ('g', 'r'), ('r', 'i'),  # Standard
        ('i', 'z'), ('z', 'y'),              # Red bands
        ('u', 'z')                           # TDE specific (UV minus IR)
    ]
    
    # DataFrame kết quả chứa object_id
    unique_ids = df["object_id"].unique()
    features = pd.DataFrame({"object_id": unique_ids})
    
    # 3. Loop qua từng cặp màu
    for f1, f2 in color_pairs:
        # Tách data theo filter
        cols = ["object_id", "Time (MJD)", "mag"]
        df1 = tmp[tmp["Filter"] == f1][cols].rename(columns={"mag": "mag1", "Time (MJD)": "time"})
        df2 = tmp[tmp["Filter"] == f2][cols].rename(columns={"mag": "mag2", "Time (MJD)": "time"})
        
        if df1.empty or df2.empty:
            continue
            
        # Sort để merge_asof
        df1 = df1.sort_values("time")
        df2 = df2.sort_values("time")
        
        # MERGE TOÀN CỤC (Global Merge) - Nhanh hơn loop từng object
        merged = pd.merge_asof(
            df1, df2,
            on="time",
            by="object_id",   
            direction="nearest",
            tolerance=1.0       
        ).dropna()
        
        if merged.empty:
            continue
            
        # Tính màu: Mag1 - Mag2
        merged["color"] = merged["mag1"] - merged["mag2"]
        
        # --- A. STATIC STATISTICS 
        # Tính Mean, Std, Min, Max cùng lúc
        aggs = merged.groupby("object_id")["color"].agg(['mean', 'std', 'min', 'max'])
        aggs.columns = [f"color_{f1}_{f2}_{stat}" for stat in aggs.columns]
        aggs = aggs.reset_index()
        
        features = features.merge(aggs, on="object_id", how="left")
        
        # --- B. DYNAMICS (Slope & Change) - Cần Apply ---
        def calc_dynamics(g):
            if len(g) < 3: 
                return pd.Series([np.nan, np.nan], index=['slope', 'change'])
            
            # 1. Slope (Tốc độ thay đổi màu)
            # TDE: Slope ~ 0 (Giữ nhiệt); SN: Slope > 0 (Nguội đi/Đỏ lên)
            try:
                # Normalize time
                t = g["time"].values - g["time"].min()
                c = g["color"].values
                slope = np.polyfit(t, c, 1)[0] 
            except:
                slope = np.nan
                
            # 2. Total Change (Cuối - Đầu)
            g_sorted = g.sort_values("time")
            # Lấy trung bình 2 điểm đầu/cuối để giảm nhiễu outlier
            start_color = g_sorted["color"].iloc[:2].mean()
            end_color = g_sorted["color"].iloc[-2:].mean()
            change = end_color - start_color
            
            return pd.Series([slope, change], index=['slope', 'change'])

        dynamics = merged.groupby("object_id").apply(calc_dynamics)
        dynamics.columns = [f"color_{f1}_{f2}_slope", f"color_{f1}_{f2}_total_change"]
        dynamics = dynamics.reset_index()
        
        features = features.merge(dynamics, on="object_id", how="left")

    return features

In [ ]:
# =========================
# SHAPE FITTING PER FILTER 
# =========================

def shape_fit_per_filter(df: pd.DataFrame, flux_col: str, cfg: Config) -> pd.DataFrame:
    if not cfg.shape_fit_per_filter:
        return pd.DataFrame({"object_id": df["object_id"].unique()})
    
    err_col = "Flux_err"
    
    priority_filters = ["g", "r", "i"]
    
    tmp = df.copy()
    # Tính SNR đúng
    tmp["SNR"] = tmp[flux_col].values / tmp[err_col].values
    
    # Get peak time per object 
    idx_peaks = tmp.groupby("object_id")[flux_col].idxmax()
    t_peak_per_obj = tmp.loc[idx_peaks, ["object_id", "Time (MJD)"]].set_index("object_id")
    
    tmp = tmp.merge(t_peak_per_obj.rename(columns={"Time (MJD)": "t_peak_obj"}), on="object_id", how="left")
    tmp["t_rel"] = tmp["Time (MJD)"] - tmp["t_peak_obj"]
    
    all_feats = []
    
    for filt in priority_filters:
        # Lọc data theo filter
        filt_data = tmp[tmp["Filter"] == filt].copy()
        if len(filt_data) == 0: continue
        
        def per_obj_filt(g: pd.DataFrame) -> pd.Series:
            mask_valid = (g["t_rel"] > 0) & (g["t_rel"] < 365) & (g[flux_col] > 0) #MAX_DECAY_WINDOW   
            after = g[mask_valid].sort_values("t_rel")
            
            pl_slope = np.nan
            
            # Cần tối thiểu 5 điểm để fit tin cậy
            if len(after) >= 5:
                t_fit = after["t_rel"].values
                f_fit = after[flux_col].values
                e_fit = after[err_col].values
                
                try:
                    log_t = np.log(t_fit)
                    log_f = np.log(f_fit)

                    weights = (f_fit / e_fit) ** 2

                    coeffs = np.polyfit(log_t, log_f, 1, w=np.sqrt(weights))
                    pl_slope = coeffs[0]
                except:
                    pass
            
            return pd.Series({f"{filt}_pl_slope": pl_slope})
        
        # Apply group
        filt_feats = filt_data.groupby("object_id").apply(per_obj_filt).reset_index()
        all_feats.append(filt_feats)
    
    # Merge results
    if len(all_feats) == 0:
        return pd.DataFrame({"object_id": df["object_id"].unique()})
    
    result = all_feats[0]
    for feat_df in all_feats[1:]:
        result = result.merge(feat_df, on="object_id", how="outer")
    
    return result

In [ ]:

from scipy.stats import entropy

def variability_features(df: pd.DataFrame, flux_col: str, cfg: Config) -> pd.DataFrame:
    
    # 1. Chọn cột Error đúng
    err_col = "Flux_err"
 
    target_filters = ['g', 'r', 'i', 'z'] 
    
    tmp = df.copy()
    
    all_feats = []

    for filt in target_filters:
        # Lọc dữ liệu theo filter
        filt_df = tmp[tmp["Filter"] == filt].copy()
        
        if len(filt_df) == 0: continue

        def per_obj_filt(g: pd.DataFrame) -> pd.Series:
            f = g[flux_col].values.astype(float)
            e = g[err_col].values.astype(float)

            mask = np.isfinite(f) & np.isfinite(e) & (e > 0)
            f, e = f[mask], e[mask]
            n = len(f)

            if n < 3:
                return pd.Series({
                    f"{filt}_stetson_j": np.nan,
                    f"{filt}_von_neumann": np.nan,
                    f"{filt}_entropy": np.nan,
                })

            w_mean = np.average(f, weights=1.0 / (e**2))
            delta = (f - w_mean) / e
            stetson_j = float(np.sum(delta[:-1] * delta[1:]) / (n - 1))

            diffs = np.diff(f)
            mean_sq_diff = float(np.mean(diffs**2))
            variance = float(np.var(f, ddof=1))
            von_neumann = mean_sq_diff / variance if variance > 0 else np.nan

            hist, _ = np.histogram(f, bins=10, density=False)
            if hist.sum() > 0:
                p = hist / hist.sum()
                flux_entropy = float(entropy(p + 1e-12))
            else:
                flux_entropy = np.nan

            return pd.Series({
                f"{filt}_stetson_j": stetson_j,
                f"{filt}_von_neumann": von_neumann,
                f"{filt}_entropy": flux_entropy
            })

        # Groupby apply
        filt_df = filt_df.sort_values(["object_id", "Time (MJD)"])
        feat_df = filt_df.groupby("object_id").apply(per_obj_filt).reset_index()
        all_feats.append(feat_df)

    # Merge lại
    if not all_feats:
         return pd.DataFrame({"object_id": df["object_id"].unique()})

    result = all_feats[0]
    for f_df in all_feats[1:]:
        result = result.merge(f_df, on="object_id", how="outer")
        
    return result

In [ ]:
# =========================
# METADATA FEATURES
# =========================

def metadata_features(log: pd.DataFrame, df_lc: pd.DataFrame, cfg: Config, is_train: bool) -> pd.DataFrame:

    # 1. Prepare Metadata
    keep_cols = ["object_id", "Z", "EBV"]
    if "target" in log.columns and is_train:
        keep_cols.append("target")
    # Giữ Z_err nếu có (quan trọng để đánh giá độ tin cậy của M_abs sau này)
    if "Z_err" in log.columns:
        keep_cols.append("Z_err")

    meta = log[keep_cols].copy()

    valid_z = (meta["Z"] > 1e-5) & np.isfinite(meta["Z"])
    meta["dL_Mpc"] = np.nan
    meta.loc[valid_z, "dL_Mpc"] = luminosity_distance_Mpc_fast(meta.loc[valid_z, "Z"].values)

    lc = df_lc.copy()
    flux_col = "Flux"
    
    peak_fluxes = lc.sort_values(flux_col, ascending=False).drop_duplicates(["object_id", "Filter"])
    
    # Pivot thành cột: peak_flux_u, peak_flux_g, ...
    peak_pivot = peak_fluxes.pivot(index="object_id", columns="Filter", values=flux_col)
    peak_pivot.columns = [f"peak_flux_{f}" for f in peak_pivot.columns]
    
    # Merge vào meta
    meta = meta.merge(peak_pivot, on="object_id", how="left")

    for f in ['g', 'r', 'u']: 
        col = f"peak_flux_{f}"
        if col in meta.columns:
            meta[f"M_abs_{f}"] = absolute_mag_from_flux_uJy(
                meta[col].values, 
                meta["dL_Mpc"].values
            )
        else:
            meta[f"M_abs_{f}"] = np.nan

    # 5. Derived Features
    meta["log_Z"] = np.log10(np.maximum(meta["Z"], 1e-6)) # Safe log
    meta["EBV_squared"] = meta["EBV"] ** 2

    return meta

In [ ]:
def advanced_physics_features(df: pd.DataFrame, flux_col: str, cfg: Config) -> pd.DataFrame:
    """
    Tính toán 4 đặc trưng vật lý cao cấp:
    1. Late Flux Fraction (Đo lường độ tàn dư)
    2. Multi-band Coherence (Sự đồng bộ giữa các dải màu)
    3. Plateau Detection (Độ phẳng tại đỉnh)
    4. Pre-explosion Variability (Độ ổn định trước khi nổ)
    """
    tmp = df.copy()
    
    # Lọc nhiễu cơ bản để tính toán sạch hơn
    # Chỉ lấy dữ liệu Valid
    tmp = tmp.dropna(subset=[flux_col, "Time (MJD)"])
    
    # Chuẩn bị DataFrame kết quả
    unique_ids = tmp["object_id"].unique()
    features = pd.DataFrame({"object_id": unique_ids})
    
    # Helper để tính correlation (Feature 2)
    def calc_coherence(sub_df):
        # Lấy band g và r (phổ biến nhất)
        g = sub_df[sub_df["Filter"] == 'g'][["Time (MJD)", flux_col]].sort_values("Time (MJD)")
        r = sub_df[sub_df["Filter"] == 'r'][["Time (MJD)", flux_col]].sort_values("Time (MJD)")
        
        if len(g) < 5 or len(r) < 5: return np.nan
        
        # Merge asof để ghép cặp thời gian gần nhất (trong vòng 1 ngày)
        merged = pd.merge_asof(g, r, on="Time (MJD)", direction='nearest', tolerance=1.0).dropna()
        
        if len(merged) < 5: return np.nan
        
        # Chuẩn hóa (Normalize)
        g_norm = (merged[f"{flux_col}_x"] - merged[f"{flux_col}_x"].mean()) / (merged[f"{flux_col}_x"].std() + 1e-6)
        r_norm = (merged[f"{flux_col}_y"] - merged[f"{flux_col}_y"].mean()) / (merged[f"{flux_col}_y"].std() + 1e-6)
        
        # Tính Correlation
        try:
            return np.corrcoef(g_norm, r_norm)[0, 1]
        except:
            return np.nan

    # Helper tính 3 feature còn lại (dựa trên Peak)
    def calc_peak_rel_features(g):
        # Tìm đỉnh
        if g[flux_col].max() <= 0: return pd.Series([np.nan]*3, index=['late_flux_ratio', 'plateau_score', 'pre_burst_std'])
        
        idx_max = g[flux_col].idxmax()
        t_peak = g.loc[idx_max, "Time (MJD)"]
        f_peak = g.loc[idx_max, flux_col]
        
        # Calculate t_rel
        t_rel = g["Time (MJD)"] - t_peak
        
        # --- 1. LATE FLUX FRACTION (Tail Energy) ---
        # TDE: Vẫn còn sáng sau 50-100 ngày
        # SN: Tắt ngấm
        late_mask = (t_rel > 50) & (t_rel < 200) # Cửa sổ 50-200 ngày sau đỉnh
        late_flux = g.loc[late_mask, flux_col]
        
        if len(late_flux) > 0:
            # Lấy median của đuôi chia cho đỉnh
            late_ratio = late_flux.median() / f_peak
        else:
            late_ratio = 0.0 # Không có dữ liệu đuôi coi như tắt ngấm
            
        # --- 3. PLATEAU DETECTION ---
        # TDE/SN IIp: Có giai đoạn phẳng quanh đỉnh
        # SN Ia: Nhọn hoắt
        # Cửa sổ: +/- 15 ngày quanh đỉnh
        plat_mask = (t_rel > -15) & (t_rel < 15)
        plat_flux = g.loc[plat_mask, flux_col]
        
        if len(plat_flux) > 3:
            # Coefficient of Variation (CV) = Std / Mean
            # Nếu phẳng -> Std thấp -> Score thấp
            plateau_score = plat_flux.std() / (plat_flux.mean() + 1e-6)
        else:
            plateau_score = np.nan
            
        # --- 4. PRE-EXPLOSION VARIABILITY ---
        # TDE: Yên tĩnh tuyệt đối trước khi nổ (Baseline Subtracted ~ 0)
        # AGN: Biến động liên tục
        pre_mask = (t_rel < -30) # Trước đỉnh 30 ngày
        pre_flux = g.loc[pre_mask, flux_col]
        
        if len(pre_flux) > 3:
            pre_burst_std = pre_flux.std()
        else:
            pre_burst_std = np.nan
            
        return pd.Series({
            'late_flux_ratio': late_ratio,
            'plateau_score': plateau_score,
            'pre_burst_std': pre_burst_std
        })

    # --- THỰC THI (Apply) ---
    # 1. Coherence (Tính trên toàn cục object, không cần groupby peak)
    print("   -> Calculating Multi-band Coherence...")
    coherence = tmp.groupby("object_id").apply(calc_coherence).reset_index(name="multiband_coherence")
    features = features.merge(coherence, on="object_id", how="left")
    
    # 2. Peak-based features
    print("   -> Calculating Peak-based Physics features...")
    # Group và apply hàm tính 3 chỉ số kia
    peak_feats = tmp.groupby("object_id").apply(calc_peak_rel_features).reset_index()
    features = features.merge(peak_feats, on="object_id", how="left")
    
    return features

In [ ]:
# =========================
# IMPUTATION + MISSING 
# =========================

def impute_features(train_df: pd.DataFrame, test_df: pd.DataFrame, 
                              target_col: str = "target") -> Tuple[pd.DataFrame, pd.DataFrame]:
    exclude = ["object_id", "split", "dataset"]
    if target_col in train_df.columns:
        exclude.append(target_col)

    all_cols = sorted(set(train_df.columns) | set(test_df.columns))
    
    train_df = train_df.reindex(columns=all_cols)
    test_df  = test_df.reindex(columns=all_cols)
    
    # Xác định danh sách feature (loại bỏ ID và Target)
    feat_cols = [c for c in all_cols if c not in exclude]

    train_df[feat_cols] = train_df[feat_cols].replace([np.inf, -np.inf], np.nan)
    test_df[feat_cols]  = test_df[feat_cols].replace([np.inf, -np.inf], np.nan)

    count_like_cols = [c for c in feat_cols if (
        c.startswith("n_") or           # n_peaks, n_obs...
        c.startswith("number_") or
        "count" in c or
        "has_" in c or                  # has_clear_peak...
        "_reliable" in c or             # decay_reliable...
        "is_" in c                      # is_missing...
    )]
    
    if count_like_cols:
        train_df[count_like_cols] = train_df[count_like_cols].fillna(0)
        test_df[count_like_cols]  = test_df[count_like_cols].fillna(0)
        logger.info(f"  Filled 0 for {len(count_like_cols)} count/logic columns.")

    critical_patterns = [
        "decay_discrimination", 
        "_med",    
        "rise_time",       
        "_slope",
        "pl_slope",
        "M_abs",
        "color_",      
        "_r2",          
        "_stetson",     
        "peak_snr"     
    ]
    
    added_indicators = []
    
    for pattern in critical_patterns:
        matching_cols = [c for c in feat_cols if pattern in c]
        
        for col in matching_cols:
            if col in train_df.columns:
                # Only add if there ARE missing values
                if train_df[col].isna().any() or test_df[col].isna().any():
                    ind_name = f"missing_{col}"
                    train_df[ind_name] = train_df[col].isna().astype(int)
                    test_df[ind_name] = test_df[col].isna().astype(int)
                    added_indicators.append(ind_name)
    
    logger.info(f"  ✓ Added {len(added_indicators)} missing indicators")

    
    remaining_nans = train_df[feat_cols].isna().sum().sum()
    logger.info(f"  ✓ Kept {remaining_nans} NaNs for LightGBM native handling")

    return train_df, test_df

In [ ]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.model_selection import StratifiedKFold
def feature_selection_cv_optimized(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    cfg: Config,
    target_col: str = "target"
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    FULLY FIXED & PRODUCTION-READY
    ================================
    1. Variance filter
    2. Smart correlation removal
    3. LightGBM CV importance
    4. Safe train/test reconstruction
    """
    
    logger.info("Starting FIXED feature selection...")
    
    id_cols = ["object_id", "split", "dataset"]
    drop_cols = [c for c in id_cols + [target_col] if c in train_df.columns]
    
    X = train_df.drop(columns=drop_cols).copy()
    y = train_df[target_col].values.astype(int)
    
    # ==================================================
    # 1. Variance Threshold
    # ==================================================
    vt = VarianceThreshold(threshold=0.0)
    mask = vt.fit(X).get_support()
    X = X.loc[:, mask]
    logger.info(f"  After variance filter: {X.shape[1]} features")
    
    # ==================================================
    # 2. SMART Correlation Filter
    # ==================================================
    logger.info(f"  Removing correlated features (>{cfg.corr_threshold})...")
    
    # Compute correlation efficiently
    X_temp = X.copy()
    X_temp['__target__'] = y
    corr_full = X_temp.corr().abs()
    
    corr_matrix = corr_full.drop('__target__', axis=1).drop('__target__', axis=0)
    corr_with_target = corr_full['__target__'].drop('__target__')
    
    upper = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )
    
    to_drop = set()
    
    for col in upper.columns:
        if col in to_drop:
            continue
        
        high_corr_cols = upper.index[upper[col] > cfg.corr_threshold].tolist()
        
        for base_col in high_corr_cols:
            if base_col in to_drop:
                continue
            
            if corr_with_target[col] >= corr_with_target[base_col]:
                to_drop.add(base_col)
            else:
                to_drop.add(col)
                break
    
    X = X.drop(columns=list(to_drop))
    logger.info(f"    → {X.shape[1]} features (dropped {len(to_drop)})")
    
    # ==================================================
    # 3. LightGBM Importance
    # ==================================================
    logger.info("  Computing LightGBM importance with 5-fold CV...")
    
    feature_names = X.columns.tolist()
    importances = np.zeros(len(feature_names))
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=cfg.random_state)
    
    for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y)):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y[tr_idx], y[va_idx]
        
        model = lgb.LGBMClassifier(
            n_estimators=200,
            learning_rate=0.05,
            num_leaves=31,
            random_state=cfg.random_state + fold,
            n_jobs=-1,
            is_unbalance=True,
            verbose=-1
        )
        
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            callbacks=[lgb.early_stopping(30, verbose=False)]
        )
        
        importances += model.booster_.feature_importance(importance_type="gain") / 5.0
    
    imp_df = pd.DataFrame({
        "feature": feature_names,
        "importance": importances
    }).sort_values("importance", ascending=False)
    
    top_features = imp_df.head(cfg.rf_top_k)["feature"].tolist()
    
    logger.info(f"  Selected top {len(top_features)} features")
    logger.info(f"  Top 5: {top_features[:5]}")
    
    # Step 4a: Align test columns with X
    test_X = test_df.drop(columns=[c for c in id_cols if c in test_df.columns]).copy()
    
    # Handle missing columns
    missing_in_test = set(X.columns) - set(test_X.columns)
    if missing_in_test:
        logger.warning(f"  ⚠ Test missing {len(missing_in_test)} cols, filling 0")
        for col in missing_in_test:
            test_X[col] = 0
    
    # Handle extra columns
    extra_in_test = set(test_X.columns) - set(X.columns)
    if extra_in_test:
        logger.warning(f"  ⚠ Test has {len(extra_in_test)} extra cols, dropping")
    
    # Ensure same order
    test_X = test_X[X.columns]
    
    # Step 4b: Select top features
    X_selected = X[top_features].reset_index(drop=True)
    test_X_selected = test_X[top_features].reset_index(drop=True)
    
    # Step 4c: Merge metadata back
    train_meta = train_df[id_cols + [target_col]].reset_index(drop=True)
    test_meta = test_df[id_cols].reset_index(drop=True)
    
    train_out = pd.concat([train_meta, X_selected], axis=1)
    test_out = pd.concat([test_meta, test_X_selected], axis=1)
    
    # Verify
    assert len(train_out) == len(train_df), "Row count mismatch!"
    assert len(test_out) == len(test_df), "Row count mismatch!"
    assert set(train_out.columns) == set(id_cols + [target_col] + top_features)
    
    return train_out, test_out

In [105]:
def load_file(cfg: Config, split_idx: int, mode: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load lightcurves and log for one split."""
    split_name = f"split_{split_idx:02d}"
    split_dir = f"cleaned_data/{split_name}"

    lc_path =  f"{split_dir}_{mode}_processed.csv"
    log_path = f"{mode}_log.csv"  # Log is at root level

    if not os.path.exists(lc_path):
        raise FileNotFoundError(f"Lightcurve file not found: {lc_path}")
    if not os.path.exists(log_path):
        raise FileNotFoundError(f"Log file not found: {log_path}")

    lc = pd.read_csv(lc_path)
    log = pd.read_csv(log_path)

    return lc, log

In [106]:
cfg = Config()
lc_train, log_train = load_file(cfg, split_idx=1, mode="test")


In [ ]:
def build_features_for_split(cfg: Config, split_idx: int, mode: str) -> pd.DataFrame:
    is_train = (mode == "train")
    
    logger.info(f"{'='*60}")
    logger.info(f"Processing split_{split_idx:02d} - {mode.upper()}")
    logger.info(f"{'='*60}")

    # Load
    lc, log_full = load_file(cfg, split_idx, mode)


    # Merge metadata
    merge_cols = ["object_id", "Z", "EBV"]
    if (not is_train) and ("Z_err" in log_full.columns):
        merge_cols.append("Z_err")
    if is_train and ("target" in log_full.columns):
        merge_cols.append("target")

    lc = lc.merge(log_full[merge_cols], on="object_id", how="left")

    flux_col = "Flux"

    # ===== LEVEL 1: Metadata =====
    logger.info("Extracting Level 1: Metadata features...")
    meta_df_source = lc.groupby("object_id").first().reset_index()
    meta = metadata_features(meta_df_source, lc, cfg, is_train=is_train)

    # ===== LEVEL 2: Statistical =====
    logger.info("Extracting Level 2: Statistical features per filter...")
    counts = count_features_per_filter(lc, cfg)
    stats_flux = stats_features_per_filter_enhanced(lc, flux_col, cfg)
    stats_err  = stats_features_per_filter_enhanced(lc, "Flux_err", cfg)

    # ===== LEVEL 3: Time-domain =====
    logger.info("Extracting Level 3: Time-domain features...")
    snr = snr_features(lc, flux_col, cfg)
    time_overall = overall_time_features_enhanced(lc, flux_col, cfg)

    # ===== LEVEL 4: Rise/Decay + Power-law =====
    logger.info("Extracting Level 4: Rise/Decay + Power-law features (CRITICAL!)...")
    rise_decay = rise_decay_features_powerlaw(lc, flux_col, cfg)

    # ===== LEVEL 5: Variability indices =====
    logger.info("Extracting Level 5: Variability indices...")
    variability = variability_features(lc, flux_col, cfg)

    # ===== LEVEL 6: Multi-band colors (UNIFIED) =====
    logger.info("Extracting Level 6: Unified Color features (Stats + Evolution)...")
    
    # Gọi hàm duy nhất này thay vì 2 hàm cũ
    colors_unified = color_features_unified(lc, flux_col, cfg)  


    # ===== LEVEL 7: Per-filter shape fitting =====
    if cfg.shape_fit_per_filter:
        logger.info("Extracting Level 7: Per-filter shape fitting...")
        shape_per_filt = shape_fit_per_filter(lc, flux_col, cfg)
    else:
        shape_per_filt = pd.DataFrame({"object_id": lc["object_id"].unique()})

    
    # ===== Combine all =====
    logger.info("Combining all features...")
    feats = meta.copy()
    
    for name, part in [
        ("counts", counts),
        ("stats_flux", stats_flux),
        ("stats_err", stats_err),
        ("snr", snr),
        ("time_overall", time_overall),
        ("rise_decay", rise_decay),
        ("variability", variability),
        ("colors", colors_unified),
        ("shape_per_filt", shape_per_filt)
    ]:
        feats = feats.merge(part, on="object_id", how="left")
        logger.info(f"  Merged {name}: {part.shape[1]-1} features")

    feats["split"] = f"split_{split_idx:02d}"
    feats["dataset"] = mode

   

    logger.info(f"✓ Completed split_{split_idx:02d}: {len(feats)} objects, {len(feats.columns)} features")

    print(feats["object_id"].nunique())
    
    return feats

In [ ]:
# =========================
# MAIN: ALL SPLITS
# =========================

def run_all(cfg: Config) -> Tuple[pd.DataFrame, pd.DataFrame]:
    os.makedirs(cfg.out_root, exist_ok=True)
    
    logger.info("="*80)
    logger.info("TDE FEATURE ENGINEERING - COMPLETE PIPELINE")
    logger.info("="*80)
    logger.info(f"Configuration:")
    logger.info(f"  Data root: {cfg.data_root}")
    logger.info(f"  Output root: {cfg.out_root}")
    logger.info(f"  Splits: {cfg.splits[0]} to {cfg.splits[1]}")
    logger.info(f"  Shape fitting: {cfg.do_shape_fitting}")
    logger.info(f"  Per-filter fitting: {cfg.shape_fit_per_filter}")
    logger.info(f"  Feature selection: {cfg.do_feature_selection}")
    logger.info(f"  Multiprocessing: {cfg.use_multiprocessing}")
    logger.info("="*80)

    if cfg.use_multiprocessing:
        # Parallel processing
        from multiprocessing import Pool
        
        logger.info(f"Using multiprocessing with {cfg.n_jobs} workers...")
        
        with Pool(processes=cfg.n_jobs) as pool:
            train_args = [(cfg, s, "train") for s in range(cfg.splits[0], cfg.splits[1] + 1)]
            test_args = [(cfg, s, "test") for s in range(cfg.splits[0], cfg.splits[1] + 1)]
            
            train_parts = pool.starmap(build_features_for_split, train_args)
            test_parts = pool.starmap(build_features_for_split, test_args)
    else:
        # Sequential processing
        train_parts = []
        test_parts = []

        for s in range(cfg.splits[0], cfg.splits[1] + 1):
            tr = build_features_for_split(cfg, s, "train")
            te = build_features_for_split(cfg, s, "test")

            train_parts.append(tr)
            test_parts.append(te)

    # Concatenate
    logger.info("Concatenating all splits...")
    train_df = pd.concat(train_parts, ignore_index=True)
    test_df = pd.concat(test_parts, ignore_index=True)
    
    logger.info(f"  Train: {len(train_df)} objects, {len(train_df.columns)} features")
    logger.info(f"  Test:  {len(test_df)} objects, {len(test_df.columns)} features")

    # Imputation
    if cfg.do_imputation:
        logger.info("Applying imputation...")
        train_df, test_df = impute_features(train_df, test_df, target_col="target")

    # Feature selection
    if cfg.do_feature_selection:
        logger.info("Applying feature selection...")
        train_df, test_df = feature_selection_cv_optimized(train_df, test_df, cfg, target_col="target")

    # Save
    train_path = os.path.join(cfg.out_root, "train_features_all_splits.csv")
    test_path = os.path.join(cfg.out_root, "test_features_all_splits.csv")
    
    train_df.to_csv(train_path, index=False)
    test_df.to_csv(test_path, index=False)

    logger.info("="*80)
    logger.info("FEATURE ENGINEERING COMPLETE!")
    logger.info("="*80)
    logger.info(f"✓ Train features: {train_path}")
    logger.info(f"  - {len(train_df)} objects")
    logger.info(f"  - {len(train_df.columns)} total columns")
    logger.info(f"  - {len([c for c in train_df.columns if c not in ['object_id', 'split', 'dataset', 'target']])} feature columns")
    logger.info(f"✓ Test features: {test_path}")
    logger.info(f"  - {len(test_df)} objects")
    logger.info(f"  - {len(test_df.columns)} total columns")
    
    # Summary statistics
    if "target" in train_df.columns:
        target_dist = train_df["target"].value_counts()
        logger.info(f"\nTarget distribution:")
        logger.info(f"  Non-TDE (0): {target_dist.get(0, 0)}")
        logger.info(f"  TDE (1):     {target_dist.get(1, 0)}")
        logger.info(f"  Ratio:       1:{target_dist.get(0, 0)/target_dist.get(1, 1):.1f}")
    
    logger.info("="*80)
    
    return train_df, test_df

In [ ]:
# =========================
# ENTRY POINT
# =========================

if __name__ == "__main__":
    cfg = Config()
    # ====== CONFIGURATION - OPTIMIZED FOR TDE CLASSIFICATION ======
          
    cfg.do_imputation = True             

    cfg.do_shape_fitting = True          
    cfg.shape_fit_per_filter = True      
    cfg.shape_fit_min_points = 6         
    cfg.shape_fit_min_peak_snr = 5.0     
    
    # Enable feature selection
    cfg.do_feature_selection = True   
    cfg.rf_top_k = 150                  
    
    cfg.use_multiprocessing = False    
    cfg.n_jobs = 4
    
    # ==============================================================

    logger.info("Starting feature engineering with OPTIMIZED configuration...")
    logger.info("Key improvements:")
    logger.info("  ✓ Power-law vs exponential decay discrimination")
    logger.info("  ✓ UV excess (u-g color)")
    logger.info("  ✓ Color evolution rates")
    logger.info("  ✓ Per-filter shape fitting")
    logger.info("  ✓ Enhanced statistical features (skew, kurt, MAD)")
    logger.info("  ✓ Variability indices (Stetson J, Von Neumann)")
    logger.info("  ✓ Improved missing data handling")
    logger.info("  ✓ CV-based feature selection")
    
    train_df, test_df = run_all(cfg)
    
    logger.info("\n" + "="*80)
    logger.info("SUCCESS! Ready for modeling.")
    logger.info("="*80)

2025-12-23 00:30:40,263 - INFO - Starting feature engineering with OPTIMIZED configuration...
2025-12-23 00:30:40,265 - INFO - Key improvements:
2025-12-23 00:30:40,266 - INFO -   ✓ Power-law vs exponential decay discrimination
2025-12-23 00:30:40,267 - INFO -   ✓ UV excess (u-g color)
2025-12-23 00:30:40,268 - INFO -   ✓ Color evolution rates
2025-12-23 00:30:40,269 - INFO -   ✓ Per-filter shape fitting
2025-12-23 00:30:40,270 - INFO -   ✓ Enhanced statistical features (skew, kurt, MAD)
2025-12-23 00:30:40,271 - INFO -   ✓ Variability indices (Stetson J, Von Neumann)
2025-12-23 00:30:40,272 - INFO -   ✓ Improved missing data handling
2025-12-23 00:30:40,273 - INFO -   ✓ CV-based feature selection
2025-12-23 00:30:40,275 - INFO - ================================================================================
2025-12-23 00:30:40,276 - INFO - TDE FEATURE ENGINEERING - COMPLETE PIPELINE
2025-12-23 00:30:40,277 - INFO - =====================================================================

155


2025-12-23 00:30:53,760 - INFO - Extracting Level 1: Metadata features...
2025-12-23 00:30:53,958 - INFO - Extracting Level 2: Statistical features per filter...
2025-12-23 00:31:08,707 - INFO - Extracting Level 3: Time-domain features...
2025-12-23 00:31:09,377 - INFO - Extracting Level 4: Rise/Decay + Power-law features (CRITICAL!)...
2025-12-23 00:31:11,618 - INFO - Extracting Level 5: Variability indices...
2025-12-23 00:31:14,363 - INFO - Extracting Level 6: Unified Color features (Stats + Evolution)...
2025-12-23 00:31:17,247 - INFO - Extracting Level 7: Per-filter shape fitting...
2025-12-23 00:31:19,288 - INFO - Combining all features...
2025-12-23 00:31:19,295 - INFO -   Merged counts: 6 features
2025-12-23 00:31:19,301 - INFO -   Merged stats_flux: 72 features
2025-12-23 00:31:19,307 - INFO -   Merged stats_err: 72 features
2025-12-23 00:31:19,313 - INFO -   Merged snr: 8 features
2025-12-23 00:31:19,320 - INFO -   Merged time_overall: 14 features
2025-12-23 00:31:19,340 - IN

364


2025-12-23 00:31:19,642 - INFO - Extracting Level 1: Metadata features...
2025-12-23 00:31:19,723 - INFO - Extracting Level 2: Statistical features per filter...
2025-12-23 00:31:28,104 - INFO - Extracting Level 3: Time-domain features...
2025-12-23 00:31:28,419 - INFO - Extracting Level 4: Rise/Decay + Power-law features (CRITICAL!)...
2025-12-23 00:31:29,173 - INFO - Extracting Level 5: Variability indices...
2025-12-23 00:31:30,403 - INFO - Extracting Level 6: Unified Color features (Stats + Evolution)...
2025-12-23 00:31:31,005 - INFO - Extracting Level 7: Per-filter shape fitting...
2025-12-23 00:31:31,937 - INFO - Combining all features...
2025-12-23 00:31:31,943 - INFO -   Merged counts: 6 features
2025-12-23 00:31:31,949 - INFO -   Merged stats_flux: 72 features
2025-12-23 00:31:31,954 - INFO -   Merged stats_err: 72 features
2025-12-23 00:31:31,961 - INFO -   Merged snr: 8 features
2025-12-23 00:31:31,967 - INFO -   Merged time_overall: 14 features
2025-12-23 00:31:31,973 - IN

...

2025-12-23 00:36:34,636 - INFO - Extracting Level 3: Time-domain features...
2025-12-23 00:36:34,726 - INFO - Extracting Level 4: Rise/Decay + Power-law features (CRITICAL!)...
2025-12-23 00:36:34,960 - INFO - Extracting Level 5: Variability indices...
2025-12-23 00:36:35,321 - INFO - Extracting Level 6: Unified Color features (Stats + Evolution)...
2025-12-23 00:36:35,480 - INFO - Extracting Level 7: Per-filter shape fitting...
2025-12-23 00:36:35,760 - INFO - Combining all features...
2025-12-23 00:36:35,762 - INFO -   Merged counts: 6 features
2025-12-23 00:36:35,764 - INFO -   Merged stats_flux: 72 features
2025-12-23 00:36:35,766 - INFO -   Merged stats_err: 72 features
2025-12-23 00:36:35,768 - INFO -   Merged snr: 8 features
2025-12-23 00:36:35,770 - INFO -   Merged time_overall: 14 features
2025-12-23 00:36:35,772 - INFO -   Merged rise_decay: 7 features
2025-12-23 00:36:35,775 - INFO -   Merged variability: 12 features
2025-12-23 00:36:35,777 - INFO -   Merged colors: 36 featu

153


2025-12-23 00:36:40,266 - INFO - Extracting Level 3: Time-domain features...
2025-12-23 00:36:40,453 - INFO - Extracting Level 4: Rise/Decay + Power-law features (CRITICAL!)...
2025-12-23 00:36:40,929 - INFO - Extracting Level 5: Variability indices...
2025-12-23 00:36:41,704 - INFO - Extracting Level 6: Unified Color features (Stats + Evolution)...
2025-12-23 00:36:42,004 - INFO - Extracting Level 7: Per-filter shape fitting...
2025-12-23 00:36:42,589 - INFO - Combining all features...
2025-12-23 00:36:42,591 - INFO -   Merged counts: 6 features
2025-12-23 00:36:42,592 - INFO -   Merged stats_flux: 72 features
2025-12-23 00:36:42,595 - INFO -   Merged stats_err: 72 features
2025-12-23 00:36:42,597 - INFO -   Merged snr: 8 features
2025-12-23 00:36:42,598 - INFO -   Merged time_overall: 14 features
2025-12-23 00:36:42,600 - INFO -   Merged rise_decay: 7 features
2025-12-23 00:36:42,602 - INFO -   Merged variability: 12 features
2025-12-23 00:36:42,604 - INFO -   Merged colors: 36 featu

358


2025-12-23 00:36:42,808 - INFO -   ✓ Kept 81343 NaNs for LightGBM native handling
2025-12-23 00:36:42,811 - INFO - Applying feature selection...
2025-12-23 00:36:42,811 - INFO - Starting FIXED feature selection...
2025-12-23 00:36:42,857 - INFO -   After variance filter: 299 features
2025-12-23 00:36:42,858 - INFO -   Removing correlated features (>0.95)...
2025-12-23 00:36:43,283 - INFO -     → 235 features (dropped 64)
2025-12-23 00:36:43,284 - INFO -   Computing LightGBM importance with 5-fold CV...
2025-12-23 00:36:46,768 - INFO -   Selected top 150 features
2025-12-23 00:36:46,769 - INFO -   Top 5: ['r_skew_Flux', 'color_g_r_max', 'g_q10_Flux', 'g_skew_Flux', 'M_abs_u']
2025-12-23 00:36:46,874 - WARNING -   ⚠ Test has 67 extra cols, dropping
2025-12-23 00:36:48,248 - INFO - ================================================================================
2025-12-23 00:36:48,249 - INFO - FEATURE ENGINEERING COMPLETE!
2025-12-23 00:36:48,250 - INFO - =================================